In [22]:
import numpy as np
import pandas  as pd
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'User5'],
    'Item1': [5, 3, 4, 3, 1],
    'Item2': [3, 1, 3, 3, 5],
    'Item3': [4, 2, 4, 1, 5],
    'Item4': [4, 3, 3, 5, 2],
    'Item5': [np.NaN, 3, 5, 4, 1],
    
}

df = pd.DataFrame(data).set_index('User')
df2 = df.fillna(0)
print(df2)
user_similarity = cosine_similarity(df2)

user_similarity_df = pd.DataFrame(user_similarity, index=df.index, columns=df.index)
print(user_similarity_df)

def predict_rating(user, item):
    # Filter out NaN values from the user similarity and item ratings
    similarity_scores = user_similarity_df[user]
    item_ratings = df[item]

    # Only consider non-NaN ratings
    mask = item_ratings.notnull() & (similarity_scores > 0)
    
    similarity_scores = similarity_scores[mask].sort_values(ascending = False)
    item_ratings = item_ratings.loc[similarity_scores.index]
    

    # If no valid similarity scores or item ratings exist, return 0
    if similarity_scores.empty or item_ratings.empty:
        return 0

    # Compute the weighted prediction
    numerator = np.sum(similarity_scores * item_ratings)
    
    denominator = np.sum(np.abs(similarity_scores))

    if denominator == 0:
        return 0
    return numerator / denominator
    
def recommended_items(user):
    unrated_items = df.columns[pd.isna(df.loc[user])]
    predictions = [predict_rating(user,item) for item in unrated_items]
    recommendations = pd.DataFrame({'Item' : unrated_items, 'Prediction' : predictions})
    return recommendations.sort_values(by = 'Prediction', ascending = False)


user_to_recommend = 'User1'
recommendations = recommended_items(user_to_recommend)
print(f"Recommendations for {user_to_recommend}")
print(recommendations)

       Item1  Item2  Item3  Item4  Item5
User                                    
User1      5      3      4      4    0.0
User2      3      1      2      3    3.0
User3      4      3      4      3    5.0
User4      3      3      1      5    4.0
User5      1      5      5      2    1.0
User      User1     User2     User3     User4     User5
User                                                   
User1  1.000000  0.826869  0.810163  0.762770  0.789542
User2  0.826869  1.000000  0.959383  0.935693  0.637815
User3  0.810163  0.959383  1.000000  0.894427  0.771517
User4  0.762770  0.935693  0.894427  1.000000  0.638311
User5  0.789542  0.637815  0.771517  0.638311  1.000000
Recommendations for User1
    Item  Prediction
0  Item5    3.252093
